In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [74]:
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [75]:
comments=pd.read_csv('comments_data/AnotherTop10CopycatMovies_comments.csv')
tran=pd.read_csv('comments_data/AnotherTop10CopycatMovies_transcript_.csv')

comments['Comment_length']=comments.Comment.apply(lambda s: s if type(s)==float else len(s))


In [7]:
comments[comments.Comment_length>200].head()

,Unnamed: 0,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes,Comment_length
26,26,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,Ok ok ok... so you guys are saying that Megami...,Ugzkvu8XxvBCkWj7bMB4AaABAg,7,75,253
70,70,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,yes Turner and Hooch and K9 have similar story...,Ugyvu7PKsWRciakbSRR4AaABAg,1,5,249


In [83]:
comments.isnull().values.any()

False

In [88]:
all_tran_text=''
for s in tran.text:
    all_tran_text+=s+' '

"hese movies gave us an uncanny sense of deja vu welcome to watchmojo.com cat movies for this list we're looking at more twin films that share similar premises or plots and came out around the same ti"

In [101]:
comments.append({'Comment':all_tran_text }, ignore_index=True)['Comment'].tail()

96               I AM \n\n\n\n\n\n\n\nThe flying Gorilla
97     How dare you compare Megamind to the nonsense ...
98     They made a third movie this year angel has fa...
99     Project X & 21 and Over. They came out roughly...
100    these movies gave us an uncanny sense of deja ...
Name: Comment, dtype: object

In [19]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [102]:
# Might take awhile...
bow_transformer = CountVectorizer(analyzer=text_process).fit(comments.append({'Comment':all_tran_text }, ignore_index=True)['Comment'])

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

1087


In [125]:
comments_bow = bow_transformer.transform(comments.append({'Comment':all_tran_text }, ignore_index=True)['Comment'])

In [157]:
print(bow_transformer.transform([' Project X  21  ']))

  (0, 9)	1
  (0, 174)	1
  (0, 222)	1


In [128]:
print('Shape of Sparse Matrix: ', comments_bow.shape)
print('Amount of Non-Zero occurences: ', comments_bow.nnz)

Shape of Sparse Matrix:  (101, 1087)
Amount of Non-Zero occurences:  1448


In [129]:
sparsity = (100.0 * comments_bow.nnz / (comments_bow.shape[0] * comments_bow.shape[1]))
print('sparsity: {}'.format(round(sparsity,3)))

sparsity: 1.319


In [130]:
tfidf_transformer = TfidfTransformer().fit(comments_bow)

In [132]:
comments_tfidf = tfidf_transformer.transform(comments_bow)
print(comments_tfidf.shape)

(101, 1087)


In [159]:
print(tfidf_transformer.transform(bow_transformer.transform([' Project X  21  '])))

  (0, 222)	0.5773502691896258
  (0, 174)	0.5773502691896258
  (0, 9)	0.5773502691896258


## testing cosine sim

In [184]:
tran_string_test=tfidf_transformer.transform(bow_transformer.transform([tran.text.iloc[200]]))
comment_string_test=tfidf_transformer.transform(bow_transformer.transform([comments.Comment.iloc[1]]))   

In [185]:
cosine_similarity(tran_string_test,comment_string_test)

array([[0.07109088]])

In [183]:
print(tran.text.iloc[200])
print(comments.Comment.iloc[1])

dreamworks Megamind arrived right on the
The biggest difference between Megamind and Despicable Me is that in Megamind the comic relief (Minion) is a million times more likeable and we can actually understand him


## create a cosine sim column for a specific tst string

In [411]:
start=60*8-8
duration=60*1-7

In [412]:
print(select_script_transcript(start,duration,tran))

call me doc number 4 Madagascar and the wild a lion and his animal buddies escaped New York's Central Park Zoo and travel by ship to Africa where they struggled to adapt to the wild this isn't so difficult this is the setup for both dreamworks madagascar and disney's the wild made during the height of their rivalry the catalyst in both computer animated fish-out-of-water comedies is a zoo animal who longs to experience mother nature madagascar beat the wild to cinemas and was a hit with both moviegoers and most critics it may be coincidence but the wild focus on the relationship between father and cub and prominently featured an active volcano all elements that would find their way into Madagascar - at least I saw the wild before it disappeared I can still see it it's right here I found my roar you deserve this son 


In [354]:
tran_string_test=tfidf_transformer.transform(bow_transformer.transform([select_script_transcript(start,duration,tran)]))

In [355]:
comments['sim_score']=comments.Comment.apply(lambda s: cosine_similarity(tran_string_test,tfidf_transformer.transform(bow_transformer.transform([s])))[0][0] )

In [352]:
print(comments.sort_values(by='sim_score',ascending=False).Comment.iloc[4])

Megamind is way better than Despicable Me


In [423]:
(comments.sort_values(by='sim_score',ascending=False).head(3).iloc[0].Replies)

2

In [427]:
i=0
while i < len(comments.sort_values(by='sim_score',ascending=False).head(3)):
    print(comments.sort_values(by='sim_score',ascending=False).head(3).iloc[i].Comment)
    i+=1

Buuuuuuutttt, both the wild and Madagascar are great movies,
The wild came out 2006 . Madagascar came out 2005 .
They made a third movie this year angel has fallen i saw it it was good at least to me.


In [432]:
comments.sort_values(by='sim_score',ascending=False).head(8)

,Unnamed: 0,Channel,Video Title,Video Description,Video ID,Comment,Comment ID,Replies,Likes,Comment_length,sim_score
37,37,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,"Buuuuuuutttt, both the wild and Madagascar are...",UgzDIagNI8ryqF6fxYZ4AaABAg,2,10,60,0.315788
34,34,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,The wild came out 2006 . Madagascar came out 2...,UgwYBXGxCr9zEc64-hl4AaABAg,2,15,51,0.253807
61,61,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,"I mean, I grew up with both Madagascar and The...",UgwKVyZuerFtgCEfeoF4AaABAg,0,2,73,0.136106
98,98,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,They made a third movie this year angel has fa...,UgzM0NAczYUgvjROeWd4AaABAg,1,2,87,0.118048
28,28,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,when I saw the title of the video say another....,Ugx5F1A9N5PWTTubeuR4AaABAg,0,12,168,0.092078
31,31,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,Tough gig for Churchill saw the trailer for th...,Ugz0Uin-mxl_dpZ2wgh4AaABAg,1,9,96,0.077401
54,54,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,I see Megamind I click,UgwXGuWPmAM3Bj46aDl4AaABAg,0,9,22,0.068053
18,18,WatchMojo.com,Another Top 10 Copycat Movies,"Talk about déjà vu. For this list, we're looki...",-VidQFzpW7M,"Watching “Megamind” still makes me laugh, it’s...",UgyvztIBfL4N6zYII2N4AaABAg,4,32,157,0.048594


## pick out a time slot from transcript

In [256]:
tran['end']=tran['start']+tran['duration']

In [304]:
def select_script_transcript(start_time,added_time,tran):
    #start_time=20
    end_time=start_time+added_time
    #a[0] <= b[0] <= a[1] or b[0] <= a[0] <= b[1]
    #tran[((start_time<=tran['start'])&(tran['start']<=end_time))]
    #tran[((tran['start']<=start_time)&(start_time<=tran['end']))]

    transelection=tran[((start_time<=tran['start'])&(tran['start']<=end_time)) | ((tran['start']<=start_time)&(start_time<=tran['end']))]

    transelection_text=''
    for s in transelection.text:
        transelection_text+=s+' '
    return(transelection_text)

In [309]:
select_script_transcript(60*5,60*2,tran)

"to strata so how did she do she a pass or fail number 6k9 and Turner and hooch a detective partners up with a troublesome dog and they bond after an initial clash of personalities release just three months apart these buddy cop action comedies share the same premise and some similar sequences including the dog downing booze seducing a lady friend and some one-sided shouting matches what the plot strikeout in different directions as Belushi's loose cannon detective pursues a drug bust with German Shepherd police dog Jerry Lee and Tom Hanks fastidious character tracks down the criminals who murdered hooch his owner but both climaxes featured classic taking the bullet moments wearing canines earn their moniker of man's best friend [Music] number five Tombstone and Wyatt Earp originally Kevin Costner was attached to Tombstone but left to make his own wide earth movie with star wars scribe Lawrence Kasdan howdy stranger we're gonna get you one of those cigars kinda nice in here that's my n

In [296]:
tran.head(9)

,Unnamed: 0,text,start,duration,end
0,0,these movies gave us an uncanny sense of,0.680,9.220,9.900
1,1,deja vu welcome to watchmojo.com cat,3.810,8.459,12.269
2,2,movies for this list we're looking at,9.900,4.380,14.280
3,3,more twin films that share similar,12.269,4.680,16.949
4,4,premises or plots and came out around,14.280,5.070,19.350
5,5,the same time a few of these even had,16.949,4.981,21.930
6,6,the same endings so beware spoilers,19.350,12.179,31.529
7,7,ahead number 10 flight plan and redeye,21.930,12.899,34.829
8,8,in both these high-altitude thrillers a,31.529,5.701,37.230


In [278]:
def overlap(a,b):
    return a[0] <= b[0] <= a[1] or b[0] <= a[0] <= b[1]

In [280]:
overlap([0,5],[102,400])

False

In [34]:
# def get_cosine_sim(vectors): 
#     vectors = [t for t in vectors]
#     return cosine_similarity(vectors)

In [43]:
cosine_similarity(np.reshape([1,0,3],(3,1)),np.reshape([0,1,0],(3,1)))

array([[0., 1., 0.],
       [0., 0., 0.],
       [0., 1., 0.]])

In [49]:
np.reshape([1,0,3],(3,1))


array([[1],
       [0],
       [3]])

In [44]:
[1,0,3]

[1, 0, 3]

In [63]:
cosine_similarity([[1,2,3]],[[0,0,1]])

array([[0.80178373]])

In [178]:
'jesse'.find('s')

2

In [182]:
tran[tran.text.str.contains('Megamind')]

,Unnamed: 0,text,start,duration
200,200,dreamworks Megamind arrived right on the,602.110,4.68
212,212,orphan children Megamind on the other,640.649,4.65


## second Try
Try again with Just a tfidf on transcript

In [428]:
tran.head(1)

,Unnamed: 0,text,start,duration,end
0,0,these movies gave us an uncanny sense of,0.68,9.22,9.9


In [429]:
def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]


bow_tran_transformer = CountVectorizer(analyzer=text_process).fit(tran['text'])

tran_bow = bow_tran_transformer.transform(tran['text'])

tfidf_tran_transformer = TfidfTransformer().fit(tran_bow)

#tran_tfidf = tfidf_tran_transformer.transform(tran_bow)


In [435]:

def select_script_transcript(start_time,added_time,tran):
    #start_time=20
    end_time=start_time+added_time
    #a[0] <= b[0] <= a[1] or b[0] <= a[0] <= b[1]
    #tran[((start_time<=tran['start'])&(tran['start']<=end_time))]
    #tran[((tran['start']<=start_time)&(start_time<=tran['end']))]

    transelection=tran[((start_time<=tran['start'])&(tran['start']<=end_time)) | ((tran['start']<=start_time)&(start_time<=tran['end']))]

    transelection_text=''
    for s in transelection.text:
        transelection_text+=s+' '
    return(transelection_text)


start=60*8-8
duration=60*1-7

# print(select_script_transcript(start,duration,tran))

tran_string_test=tfidf_tran_transformer.transform(bow_tran_transformer.transform([select_script_transcript(start,duration,tran)]))



comments['sim_score']=comments.Comment.apply(lambda s: cosine_similarity(tran_string_test,tfidf_tran_transformer.transform(bow_tran_transformer.transform([s])))[0][0] )

relevant_comments_df=comments.sort_values(by='sim_score',ascending=False).head(3)

In [441]:
print(relevant_comments_df.Comment.iloc[0])

Buuuuuuutttt, both the wild and Madagascar are great movies,
